In [1]:
%pip install --upgrade pip
%pip install --upgrade setuptools
%pip install fpdf2
%pip install httpx aiofiles
%pip install selenium bs4 httpx aiofiles

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import httpx
from bs4 import BeautifulSoup

# Initialize Selenium WebDriver
driver = webdriver.Chrome()

def get_sub_urls(startup_url):
    """Use Selenium to collect sub-URLs from a startup's main page."""
    sub_urls = set()
    driver.get(startup_url)
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'a')))
        links = driver.find_elements(By.TAG_NAME, 'a')
        for link in links:
            href = link.get_attribute('href')
            if href and href.startswith(('http://', 'https://')):
                sub_urls.add(href)
    except Exception as e:
        print(f"Error while collecting sub-URLs from {startup_url}: {e}")
    return sub_urls

def scrape_and_clean_content(url):
    """Scrape content from a URL and clean it."""
    try:
        response = httpx.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        cleaned_text = soup.get_text(separator=' ', strip=True).replace('\n', ' ').replace('\t', ' ')
        return cleaned_text
    except Exception as e:
        print(f"Error while scraping {url}: {e}")
        return ""

def process_startups(csv_file_path):
    """Process each startup: scrape sub-URLs, clean content, and save."""
    with open(csv_file_path, mode='r', newline='', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            startup_name = row['startup_name']
            startup_url = row['startup_url']
            
            # Get sub-URLs using Selenium
            sub_urls = get_sub_urls(startup_url)
            
            # Initialize a list to hold cleaned content from each sub-URL
            cleaned_contents = []
            
            # Scrape and clean content from each sub-URL
            for sub_url in sub_urls:
                cleaned_content = scrape_and_clean_content(sub_url)
                if cleaned_content:
                    cleaned_contents.append(cleaned_content)
            
            # Save the cleaned content to a file named after the startup
            filename = f"{startup_name.replace(' ', '_')}.txt"
            with open(filename, mode='w', encoding='utf-8') as output_file:
                output_file.write("\n\n".join(cleaned_contents))
    
    # Close the WebDriver
    driver.quit()

# Specify the path to your CSV file
csv_file_path = "/home/lenovo/Desktop/scrapping/scrappa.csv"
process_startups(csv_file_path)



Error while scraping https://locusrobotics.com/solutions/warehouse-transport/: Client error '403 Forbidden' for url 'https://locusrobotics.com/solutions/warehouse-transport/'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Error while scraping https://locusrobotics.com/industries/retail-and-ecommerce-fulfillment/: Client error '403 Forbidden' for url 'https://locusrobotics.com/industries/retail-and-ecommerce-fulfillment/'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Error while scraping https://locusrobotics.com/solutions/mezzanine-warehouse-management/: Client error '403 Forbidden' for url 'https://locusrobotics.com/solutions/mezzanine-warehouse-management/'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Error while scraping https://locusrobotics.com/locusone/automated-warehouse-software/multi-robot: Client error '403 Forbidden' for url 'https://locusrobotics.